In [5]:
from lxml import etree
import itertools
import roman, textwrap

vimmXML='build/vimm7a.exported.xml'
# vimmBook=etree.parse(vimmXML0,etree.XMLParser(remove_blank_text=True)).getroot() 

book=etree.parse('src/book.6.xml',etree.XMLParser(remove_blank_text=True)).getroot() 

for inc in book.findall('.//include'):
    incRoot=etree.parse('src/'+inc.attrib['file'],etree.XMLParser(remove_blank_text=True,remove_comments=True)).getroot()
    inc.getparent().replace(inc,incRoot)
    
def add_sections(book,tocXML,vimm=False):
    flatBook=list(book.iter())
    # hand-tuned toc, do not overwrite
    toc=etree.parse(tocXML,etree.XMLParser(remove_comments=True)).getroot() 

    def _one_only(pp):
        assert(len(pp)==1)
        return pp[0]

    for toc_chap in toc:
        chap_num=toc_chap.attrib['num']
        print(chap_num)
        chap=_one_only(book.findall('.//heading[@toc_num="'+chap_num+'"]')).getparent()
        if not vimm:
            chapLastPara=str(max([int(e.attrib['num']) for e in chap if e.tag=='p' and 'num' in e.attrib]))
            toc_chap.attrib['para_last']=chapLastPara
        else:
            elem0=chap[0]
        ee0={}
        # find first DOM element for each sectioning piece
        for toc_sect in toc_chap.iter():
            if toc_sect.tag=='chapter': continue
            try: para,title,starts_at=toc_sect.get('para',None),toc_sect.attrib['title'],toc_sect.attrib.get('starts_at',None)
            except KeyError:
                print(f'Missing keyword? line {toc_sect.sourceline} {toc_sect.tag}')
                raise
            if not vimm: # Visuddhimagga
                if para is None: raise RuntimError(f'Missing para for Visuddhimagga {toc_sect.sourceline} {toc_sect.tag}')
                if len(pp:=list(chap.findall('.//p[@num="'+para+'"]')))!=1:
                    print(f'{chap_num} {para=} {starts_at=}')
                elem0=_one_only(chap.findall('.//p[@num="'+para+'"]'))
            if starts_at is not None:
                for ix in itertools.count(flatBook.index(elem0)+1):
                    if ix>=len(flatBook): raise RuntimeError(f'No match for "{starts_at}".')
                    e=flatBook[ix]
                    if e.tag not in ('span','em'): continue
                    if not e.text.startswith(starts_at): continue
                    parent=e.getparent()
                    if parent.index(e)>0: raise RuntimeError(f'start_at matched on {e.tag} (line {e.sourceline}) but it is not the first child of its parent <{parent.tag}> (line {parent.sourceline}) — not splitting paragraphs.')
                    if parent.tag=='p': pass
                    elif parent.tag=='line':
                        parent=parent.getparent()
                        assert parent.tag=='verse'  
                    else: raise RuntimeError(f'Matched content trouble: parent of {e.tag} (line {e.sourceline}) must be <p> or <line> (not {parent.tag}).')
                    elem0=parent
                    break
                # if elem0 is None: raise RuntimeError(f'start_at did not match anything (chapter {chap_num}, line {e.sourceline}, {starts_at=})')
            ee0[toc_sect]=elem0
            assert elem0.getparent().tag=='struct-2-chapter'
            if vimm: elem0=flatBook[flatBook.index(elem0)+1]
        def _subdivide(sects,level,lastPara):
            sect=None
            for isect,sect in enumerate(sects):
                # print(sect.sourceline)
                l2=level+3
                new=etree.Element(f'struct-{l2}-{level*"sub"}section')
                new.append(etree.Element(f'heading')) # -{l2}-{level*"sub"}section'))
                new[-1].text=sect.attrib['title']
                new.attrib['par_begin']=sect.attrib['para']
                if isect==len(sects)-1:
                    eAfter=None
                    new.attrib['par_end']=sect.attrib['para_end']=lastPara
                else:
                    sAfter=sects[isect+1]
                    eAfter=ee0[sAfter]
                    new.attrib['par_end']=sect.attrib['para_end']=str(int(sAfter.attrib['para'])-(0 if 'starts_at' in sAfter.attrib else 1))
                e=ee0[sect]
                e.addprevious(new)
                pnum=new.getparent().get('sect_num',None)
                new.attrib['sect_num']=f'{pnum}.{isect+1}' if pnum else f'{isect+1}'
                while True:
                    ee=e.getnext()
                    new.append(e)
                    e=ee
                    if e==eAfter or e is None: break
                _subdivide(sect,level=level+1,lastPara=new.attrib['par_end'])
            return [sect]
        _subdivide(toc_chap,level=0,lastPara=chapLastPara)
    return toc

toc=add_sections(book,'src/toc.xml')
# add_sections(book,'vimm/toc.xml')

# re-apply to the book
#open('xml/book.7.xml','w').write(etree.tostring(book,encoding='unicode',pretty_print=True))
open('build/book.sectioned.xml','w').write(etree.tostring(book,encoding='unicode',pretty_print=True))
open('build/toc.more.xml','w').write(etree.tostring(toc,encoding='unicode',pretty_print=True))
#vimmXML='build/vimm.sectioned.xml'
#open(vimmXML,'w').write(etree.tostring(book,encoding='unicode',pretty_print=True))


I
II
III
IV
V
VI
VII
VIII
IX
X
XI
XII
XIII
XIV
XV
XVI
XVII
XVIII
XIX
XX
XXI
XXII
XXIII


34471

In [2]:
import git, urllib.parse
vismCommit=(head:=git.Repo(search_parent_directories=True).head).object.hexsha[:7]
vismCommitTimestampQuery=urllib.parse.quote('in version '+vismCommit+' dated '+head.commit.committed_datetime.date().isoformat())

In [3]:
import os, re
##
## LaTeX output
##
book=tree=etree.parse('build/book.sectioned.xml',etree.XMLParser(remove_comments=True)).getroot() 
index=tree=etree.parse('src/index.xml',etree.XMLParser()).getroot() 
gloss=tree=etree.parse('src/gloss.xml',etree.XMLParser()).getroot() 

#editorial_titles=False
#vimm=False
#mainmatter=False

class LatexWriter(object):
    def __init__(self,vimm=False,editorial_titles=False):
        self.vimm=vimm
        self.editorial_titles=editorial_titles
        self.bookMatter=-1
        self.lev=0
    # def _latex_writer(e,lev=0,ord=-1):
    def _rep(self,t): return t.replace('&','\\&').replace('#','\\#').replace('ṁ','\\.{m}').replace('ḣ','\\.{h}').replace('ḳ','\\d{k}').replace('ṡ','\\.{s}').replace('ẏ','\\.{y}')
    def _recurse(self,e):
        self.lev+=1
        if e.text is not None and len(e)==0: ret=self._rep(e.text)
        else: ret=''.join([self.write(e2,ord=ord) for ord,e2 in enumerate(e)])
        self.lev-=1
        return ret
    def _nobraces(self,t):
        return t.replace('[','').replace(']','')
    def _title(self,sect,e):
        t=self._recurse(e)
        if sect in ('chapter','part'):
            if 'subtitle_pali' not in e.attrib: return f'\\{sect}'+'{'+t+'}'
            else: return f'\\{sect}[{t}]{{{t}\\newline{{\\textnormal{{\emph{{{e.attrib["subtitle_pali"]}}}}}}}}}'
        struct=e.getparent()
        if 'par_begin' in struct.attrib:
            p0,p1=int(struct.attrib['par_begin']),int(struct.attrib['par_end'])
            if p0==p1: t2=f'§{p0}'
            else: t2=f'§{p0}–{p1}'
            # return '\\'+sect+'[\\protect\\numberline{'+t2+'}'+t+']{\\ifplastex\\else\\kern-1em\\fi{}'+t+'}'
            return '\\'+sect+'[\\vismAlignedParas{'+t2+'}'+t+']{'+t+'}' #'\\ifplastex\\else\\hfill '+t2+'\\fi}'
            # return '\\def\\vismNextParRange{'+t2+'}\\'+sect+'{'+t+'}'
        else: return '\\'+sect+'{'+t+'}'
    
    def write(self,e,ord=-1):
        ret=''
        ind=2*self.lev*'  '

        if isinstance(e,etree._Comment): return ''
        if e.tag=='book': return self._recurse(e)
        elif e.tag=='em':
            if len(e)>0: raise RuntimeError(f'<em> with child elements, line {e.sourceline}')
            if e.text is None: return ''
            return '\\emph{'+self._rep(e.text)+'}'
        elif e.tag=='span':
            if len(e)>0: raise RuntimeError(f'<span> with child elements, line {e.sourceline}')
            if e.text is None: return ''
            tx=self._rep(e.text)
            if (fam:=e.attrib.get('family',None)) is None: return tx
            elif fam=='italic': return '\\emph{'+tx+'}'
            elif fam=='bold': return '\\textbf{'+tx+'}'
            elif fam=='smallcaps': return '\\textsc{'+tx+'}'
            elif fam=='bold-italic': return '\\textbf{\\emph{'+tx+'}}'
            else: raise RuntimeError(f'Unrecognized family {fam}')
        elif e.tag=='p':
            if not self.editorial_titles and 'editorial_title' in e.attrib: return ''
            ret='\n\n'+ind if ord>0 else ''
            if 'anchor' in e.attrib:
                num,anchor=e.attrib['num'],e.attrib['anchor']
                # ret+='\\paragraph{§'+num+'.}\\vismHypertarget{'+anchor+'}{}\\marginnote{\\footnotesize\\textcolor{purple}{'+anchor+'}}{}\n'+ind
                ret+='\\vismParagraph{'+anchor+'}{'+num+'}{}\n'+ind
            return ret+self._recurse(e)
        elif e.tag=='footnote':
            check=r'\vismAssertFootnoteCounter{'+e.attrib['mark']+'}'
            if 'reference_existing_footnote' in e.attrib: return check+r'\footnotemark[\value{footnote}]'
            # elif anchor:=e.attrib.get('anchor',None): return '\\footnote{'+check+'\\vismHypertarget{'+anchor+'}{}\\marginnote{\\footnotesize\\textcolor{purple}{'+anchor+'}}'+self._recurse(e)+'}'
            elif anchor:=e.attrib.get('anchor',None): return '\\footnote{'+check+'\\vismHypertarget{'+anchor+'}{}'+self._recurse(e)+'}'
            else: return '\\footnote{'+check+self._recurse(e)+'}'
        elif e.tag=='verse':
            assert e[-1].tag=='line'
            e[-1].attrib['last-line']="1"
            return '\n'+ind+'\\begin{verse}\n'+self._recurse(e)+ind+'\\end{verse}\n'
        elif e.tag=='line':
            if len(e)==0: return ind+'\n'
            return ind+self._recurse(e)+(r'\\{}' if not 'last-line' in e.attrib else '')+'\n'
        elif e.tag.startswith('heading'): return ''
        elif e.tag.startswith('struct-'):
            level,tail=int((m:=re.match('struct-(?P<tail>(?P<level>[0-9]+)-.*)',e.tag)).group('level')),m.group('tail')
            heading=e[0]
            assert heading.tag.startswith('heading')
            toc_num=heading.attrib.get('toc_num',None)
            def _recurse_wrap(e2):
                if e2.get('hanging',None): return r'\begin{vismHanging}'+self._recurse(e2)+'\end{vismHanging}'
                else: return self._recurse(e2)
            if level==1: # part
                ret=ind
                if not self.vimm:
                    frontmatter=((name:=e.attrib['name'])=='(Front)')
                    # if frontmatter: ret+='\\frontmatter'
                    if not frontmatter:
                        ret+='\n'+ind+self._title('part',heading) # no heading for frontmatter
                        if toc_num: ret+='\n'+ind+'    \\label{'+toc_num+'}\n'+ind+'    \\vismHypertarget{'+toc_num+'}\n'
                ret+=_recurse_wrap(e)
                 # Vissudhimagga: appendix after Part III
                if not self.vimm:
                    if frontmatter: ret+='\n\n'+ind+'\\mainmatter'
                    if name=='Part III': pass #   ret+='\n\n'+ind+'\\appendix'
                else:
                    self.bookMatter+=1
                    if self.bookMatter==0:
                        ret+='\n\\mainmatter\\vismUnnumberedPart{The Path of Freedom (\\emph{Vimuttimagga})}{The Path of Freedom\\\\ \\emph{Vimuttimagga}}\n'
                    if self.bookMatter==1: ret+='\n\n'+ind+'\\backmatter'
                return ret
            elif level==2: # chapter
                # \label is just for PlasTeX which can then name the output file accordingly (the chapter has an $id)
                ret='\n'+ind+self._title('chapter',heading)
                if toc_num: ret+='\n'+ind+'    \\label{'+toc_num+'}\n'+ind+'    \\vismHypertarget{'+toc_num+'}\n'
                return ret+_recurse_wrap(e)
                # else: return '\n'+ind+_title('chapter',heading)+_recurse(e)
            elif level==3: ret='\n'+ind+self._title('section',heading)
            elif level==4: ret='\n'+ind+self._title('subsection',heading)
            elif level==5: ret='\n'+ind+self._title('subsubsection',heading)
            elif level==6: ret='\n'+ind+r'\par\noindent[\textsc{\textbf{'+self._rep(heading.text)+'}}]'
            elif level==7: ret='\n'+ind+r'\par\noindent[\emph{\textbf{'+self._rep(heading.text)+'}}]'
            elif level==8: ret='\n'+ind+r'\par\noindent[\emph{'+self._rep(heading.text)+'}]'
            else: raise RuntimeError(f'Unknown {level=}')
            # levels 3+
            return ret+_recurse_wrap(e)
        elif e.tag=='printed_page':
            #if e.attrib['edition']=='BPS2011': return r'{\small\textbf{\href[page='+e.attrib['page_id']+']{PathofPurification2011.pdf}{\{'+e.text+' ('+e.attrib['page_id']+')\}}}}' # marginpar{['+e.text+r']}'
            if e.attrib['edition'] in ('BPS2011','BPS1995'):
                # return r'\marginnote[\footnotesize\{'+e.text+'('+e.attrib['page_id']+r')\}]{}[-1ex]' # this is too complicated for PlasTeX
                return r'\marginnote{\textcolor{teal}{\footnotesize\{'+e.text+'('+e.attrib['page_id']+r')\}}}{}'
            elif e.attrib['edition']=='PTS': return r'\textcolor{brown}{\textit{['+e.text+']}}'
            assert False
        elif e.tag=='ref':
            if e.attrib['type']=='vism': return r'\hyperlink{'+e.attrib['target']+r'}{'+e.text+'}{}'
            elif e.attrib['type']=='bib': 
                ret=r'\textbf{\cite{'+e.attrib['target']+'}'
                if loc:=e.get('loc',None):
                    if href:=e.get('href',None): ret+=' \href{'+self._rep(href)+'}{'+loc+'}'
                    else: ret+=loc
                return ret+'}'
            # r'\fbox{'+e.text+'→'+e.attrib['target']+'}'
            assert False
        elif e.tag in ('index','glossary'):
            title,subtitle=self._rep(e.attrib['title']),self._rep(e.attrib.get('subtitle',None))
            ret='\\chapter['+title+']{'+title+'\\* {\large '+subtitle+'}}'
            if ii:=e.findall('introductory'):
                assert len(ii)==1
                ret+=self._recurse(ii[0])
            ret+=r'\begin{multicols}{2}'+'\n'+r'\parskip=.2\baselineskip\RaggedRight\begin{vismHanging}'+'\n'+self._recurse(e)+r'\end{vismHanging}\end{multicols}'
            return ret
        elif e.tag=='introductory': return '' # already handled in index/glossary
        elif e.tag=='entry':
            assert e.getparent().tag in ('index','glossary')
            return r'\par\textbf{'+self._rep(e.attrib['title'])+'} '+self._recurse(e)+'\n'
        elif e.tag=='table':
            tbody='    '+'\\\\\n    '.join([' & '.join([self._recurse(td) for td in tr]).strip() for tr in e])+'\n'
            nRows,nCols=len(e),len(e[0])
            assert 'layout_type' in e.attrib,f'<table> without layout_type at line {e.sourceline}'
            layout_type=e.attrib['layout_type']
            if layout_type=='ceylon':
                # env,lCols,xCols='l|l|l','X[2,l]|X[4,l]|X[3,l]'
                plastex='\\begin{tabular}{l|l|l}\n'+tbody+'\\end{tabular}'
                latex='\\begin{longtblr}{colspec={X[2,l]|X[4,l]|X[3,l]}}\n'+tbody+'\\end{longtblr}'
            elif layout_type=='commentaries':
                plastex='\\begin{tabular}{ll}\n'+tbody+'\\end{tabular}'
                latex='\\begin{tblr}{colspec={Q[15em]Q[15em]}}\n'+tbody+'\\end{tblr}'
            elif layout_type=='consciousness':
                plastex='\\begin{tabular}{rrl}\n'+tbody+'\\end{tabular}'
                latex='\\begin{longtblr}[theme=vismNaked,presep=\smallskipamount,postsep=\smallskipamount]{colspec={X[1,r]Q[4em,r]X[1,l]},rowsep=0pt}\n'+tbody+'\\end{longtblr}'
            else: raise InvalidValue(f'latex_theme must be one of: ceylon, commentaries, consciousness (not {latex_theme}, XML line {e.sourceline}).')
            return textwrap.indent(f'\n\n\\ifplastex\n{plastex}\n\\else\n{latex}\\fi\n\\noindent\n',ind)+ind
        elif e.tag=='enum':
            labelSpec='label='+{'1.':'\\arabic*.','I.':'\\Roman*.','i.':'\\roman*.','(i)':'(\\roman*)','(a)':'(\\alph*)'}[e.get('labels','1.')]
            labelSpecShort=e.get('labels','1.')
            start=e.get("start","1")
            ret='\n\\begin{enumerate}['+labelSpecShort+',nosep'+(f',start={start}' if start!="1" else '')+']'
            for li in e:
                assert li.tag=='li'
                ret+='\n    \\item '+self._recurse(li)
            ret+='\n\\end{enumerate}'
            return textwrap.indent(ret,ind)
        elif e.tag=='bibliography':
            ret='\n'+ind+'\\begin{thebibliography}{xxxxxxxxxxx}'
            for bi in e:
                if bi.tag=='bibentry':
                    abbr=bi.get('abbrev')
                    ret+='\n'+ind+f'  \\bibitem[{abbr}]{{{abbr}}}'+self._recurse(bi)
                elif bi.tag=='bibintertitle':
                    assert len(bi)==0 and bi.text
                    ret+='\n'+ind+'\\vismBibIntertitle{'+bi.text+'}'
                else: assert False
            return ret+'\n'+ind+'\\end{thebibliography}\n'
        elif e.tag=='raw':
            return open('latex/'+e.attrib['file']+'.tex','r').read()
        elif e.tag=='TODO':
            return r'\textbf{[TODO: '+e.text+']}'
        elif e.tag=='IGNORE': return ''
        raise RuntimeError(f'Unhandled tag <{e.tag}>')

os.makedirs('build/latex',exist_ok=True)

vismCommitTimestampQuery_=vismCommitTimestampQuery.replace("%","\\%")
open('build/latex/vism-defs.tex','w').write(f'''
    \\def\\vismCommit{{{vismCommit}}}\n
    \\def\\vismCommitTimestampQuery{{{vismCommitTimestampQuery_}}}\n
''')


writerVism=LatexWriter()

print('BODY')
open('build/latex/vism-body.tex','w').write(writerVism.write(book))
print('INDEX')
open('build/latex/vism-index.tex','w').write(writerVism.write(index))
print('GLOSSARY')
open('build/latex/vism-glossary.tex','w').write(writerVism.write(gloss))

print('Vimuttimagga')
writerVimm=LatexWriter(vimm=True)
vimmBook=etree.parse(vimmXML,etree.XMLParser(remove_comments=True)).getroot() 
open('build/latex/vimm-body.tex','w').write(writerVimm.write(vimmBook))

    

BODY
INDEX
GLOSSARY
Vimuttimagga


1279645

In [4]:
from lxml import etree
import string

book=tree=etree.parse('build/book.sectioned.xml',etree.XMLParser()).getroot()
index=tree=etree.parse('src/index.xml',etree.XMLParser()).getroot()
gloss=tree=etree.parse('src/gloss.xml',etree.XMLParser()).getroot()

book=tree=etree.parse('build/book.sectioned.xml',etree.XMLParser()).getroot()

class SphinxWriter(object):
    def __init__(self,outdir,vimm=False):
        self.footnotes={}
        self.outdir=outdir
        self.chapter=0
        self.part=-1
        self.editorial_titles=False
        self.vimm=vimm
    def _flush(self):
        if not self.footnotes: return ''
        ret='\n\n.. rubric:: Footnotes\n\n'
        for k,vv in self.footnotes.items(): ret+=f'\n\n.. _{self.chapter_anchor}.n{k}:\n\n.. [#{k}] '+'\n    '.join([v for v in vv.split('\n')])+'\n'
        self.footnotes={}
        return ret
    def _rep(self,t): return t.replace('*','∗')
    def recurse(self,e):
        if e.text is not None and len(e)==0: return self._rep(e.text)
        return ''.join([self.write(e2,ord=ord) for ord,e2 in enumerate(e)])
    def title(self,e,level,anchor=None,prefix=None,prefixSep='. '):
        ret=''
        if anchor: ret+='\n\n.. _'+anchor+':'
        t=(e if isinstance(e,str) else self.recurse(e))
        if prefix: t=prefix+prefixSep+t
        return ret+'\n\n'+t+'\n'+len(t)*('#*=-^"\''[level])
    def enclose(self,t,c):
        if t.strip()=='': return ' '
        ret=t
        if ret.endswith(' '): ret=ret.rstrip()+c+' '
        else: ret=ret+c+'\\ '
        if ret.startswith(' '): ret=' '+c+ret.lstrip()
        else: ret=c+ret
        return ret

    def write(self,e,ord=-1):
        def _nobraces(t):
            return t.replace('[','').replace(']','')
        if isinstance(e,etree._Comment): return''
        elif e.tag=='em':
            assert len(e)==0
            if e.text is None: return ''
            return self.enclose(self._rep(e.text),'*')
        elif e.tag=='span':
            assert len(e)==0
            if e.text is None: return ''
            tx=self._rep(e.text)
            if (fam:=e.attrib.get('family',None)) is None: return tx
            elif fam=='italic': return self.enclose(tx,'*')
            elif fam=='bold': return self.enclose(tx,'**')
            elif fam=='smallcaps': return self.enclose(tx,'``')
            elif fam=='bold-italic': return self.enclose(tx,'``')
            else: raise RuntimeError(f'Unrecognized family {fam}')
        elif e.tag=='p':
            if not self.editorial_titles and 'editorial_title' in e.attrib: return ''
            if anchor:=e.attrib.get('anchor',None):
                pre=f'\n\n.. _{anchor}:\n\n`§{e.attrib["num"]} <https://github.com/eudoxos/vism/issues/new?title=issue%20at%20{anchor}&body=({vismCommitTimestampQuery})>`__ '
                return pre+self.recurse(e)
            else:
                pre=('\n\n' if ord>0 else '')
                tail=self.recurse(e)
                # fix what looks like start of enumeration but is not
                if len(e)>0 and e[0].tag=='span' and re.match('\([0-9]+\) |[0-9]+\. |[a-zA-Z]\. |\([a-zA-Z]\) ',tail): tail='\\'+tail
                return pre+tail
        elif e.tag=='footnote':
            anchor=e.attrib.get("anchor",str(len(self.footnotes)+1))
            if self.vimm:
                if len(self.footnotes)==0: mark="1"
                else: mark=str(int(list(self.footnotes.keys())[-1])+1)
            else:
                mark=e.attrib['mark']
                if 'reference_existing_footnote' in e.attrib:
                    return f' [#{mark}]_'
            self.footnotes[mark]=self.recurse(e)
            return f' [#{mark}]_ '
        elif e.tag=='verse':
            return '\n\n'+self.recurse(e)
        elif e.tag=='line': return ('\n\n' if ord==0 else '')+'\n| '+self.recurse(e)+('\n' if 'last-line' in e.attrib else '')
        elif e.tag.startswith('heading'): return ''
        elif e.tag.startswith('struct-'):
            level,tail=int((m:=re.match('struct-(?P<tail>(?P<level>[0-9]+)-.*)',e.tag)).group('level')),m.group('tail')
            heading=e[0]
            assert heading.tag.startswith('heading')
            toc_num=heading.attrib.get('toc_num',None)
            assert 0<level<9
            if level==1: # part
                self.part+=1
                if self.part==1: # begin mainmatter
                    self.chapter=0
                # Vimuttimagga: reset numbering for appendices
                if self.vimm and self.part==2:
                    self.chapter=0
                if self.vimm: f=self.outdir+'/'+['front','text','back'][self.part]+'.rst'
                else: f=self.outdir+'/'+['front','part-I','part-II','part-III'][self.part]+'.rst'
                self.partOut=open(f,'w')
                print(f'→ {f}')
                if toc_num:
                    toc_num='part-'+toc_num
                    self.partOut.write(self.title(heading,level=1,anchor=toc_num,prefix=heading.attrib["toc_name"])+'\n\n')
                # self.partOut.write('.. toctree::\n   :numbered:\n   :maxdepth: 6\n\n')
                self.partOut.write('.. toctree::\n   :maxdepth: 6\n\n')
                for chap in e:
                    self.partOut.write('\n   '+self.write(chap))
                self.partOut.close()
                return None
            elif level==2:
                self.chapter+=1
                if self.part==0: # frontmatter
                    f=f'front-{"_ABCDEFGHIJKLMN"[self.chapter]}.rst'
                else: f=f'ch-{self.chapter:02d}.rst'
                if self.vimm and self.part==2: f=f'back-{"_ABCDEFGHI"[self.chapter]}.rst'
                ff=f'{self.outdir}/{f}'
                print(f'   → {ff}')
                out=open(ff,'w')
                if self.vimm:
                    import roman
                    if self.part==1: toc_num=roman.toRoman(self.chapter)
                    elif self.part==2:
                        # print(f'{self.chapter=}')
                        toc_num=None # 'app'+'[self.chapter]
                self.chapter_anchor=toc_num
                ## TODO: e.attrib['subtitle_pali']
                out.write(self.title(heading,level=2,anchor=toc_num,prefix=toc_num))
                out.write(self.recurse(e)+self._flush())
                return f
            elif 3<=level<=6:
                # print(heading.sourceline,heading)
                return self.title(heading,prefix=e.attrib.get('sect_num',None),prefixSep=' ',level=level)+'\n\n'+self.recurse(e)
            else:
                dd={7:'**',8:'*'}[level]
                return '\n\n'+dd+'['+heading.text+']'+dd+'\ '+'\n\n'+self.recurse(e)
        elif e.tag=='book':
            for e2 in e: self.write(e2)
            return None
        elif e.tag=='printed_page':
            if e.attrib['edition'] in ('BPS2011','BPS1995'): return f'*[{e.text}/{e.attrib["page_id"]}]* '
            elif e.attrib['edition']=='PTS': return f' ``{e.text}`` '
            assert False
        elif e.tag=='ref':
            if e.attrib['type']=='vism':
                return f':ref:`{e.text} <{e.attrib["target"]}>`'
            elif e.attrib['type']=='bib':
                ret=f' [{e.attrib["target"].replace(".","")}]_ '
                if loc:=e.get('loc',None):
                    if href:=e.get('href',None): ret+=f'`{loc.strip()} <{href}>`__'
                    else: ret+=self.enclose(loc,'*')
                #+(self.enclose(e.attrib["loc"],'*') if 'loc' in e.attrib else '')+' '
                return ret+' '
            assert False
        elif e.tag in ('index','glossary'):
            title,subtitle=e.attrib['title'],e.attrib['subtitle']
            if e.tag=='index':      out,ret='index_',self.title(e=f'{title} ({subtitle})',level=1,anchor='index')
            elif e.tag=='glossary': out,ret='glossary',self.title(e=f'{title} ({subtitle})',level=1,anchor='glossary')
            if ii:=e.findall('introductory'):
                assert len(ii)==1
                ret+='\n\n'+self.recurse(ii[0])
            ret+='\n\n.. glossary::'
            ret+=self.recurse(e)
            f=f'{self.outdir}/{out}.rst'
            print(f'→ {f}')
            open(f,'w').write(ret)
            return
        elif e.tag=='introductory': return ''
        elif e.tag=='entry':
            title=e.attrib["title"].replace("*","\\*")
            return f'\n\n   {title}\n          '+self.recurse(e)
        elif e.tag=='raw':
            return '\n\n'+open('sphinx/'+e.attrib['file']+'.rst','r').read()+'\n\n'
        elif e.tag=='TODO':
            return f'**TODO: {e.text}**\ '
        elif e.tag=='IGNORE': return ''
        elif e.tag=='table':
            layout_type=e.attrib['layout_type']
            ret='\n\n.. list-table::\n'
            if layout_type=='commentaries': ret+='  :header-rows: 1\n\n'
            elif layout_type=='ceylon': ret+='  :header-rows: 1\n  :widths: 30 20 40\n\n'
            elif layout_type=='consciousness': ret+='  :width: 80%\n  :widths: 4 1 4\n\n'
            for irow,tr in enumerate(e):
                assert tr.tag=='tr'
                for icol,td in enumerate(tr):
                    assert td.tag=='td'
                    cont=self.recurse(td)
                    # escape (xiv) which would be turned into 'xiv.' (as enumeration start)
                    if icol==1 and cont[0]=='(': cont='\\'+cont
                    ret+='\n  '+('*' if icol==0 else ' ')+' - '+cont
            return ret+'\n'
        elif e.tag=='enum':
            import roman
            iOff=int(e.get('start','1'))
            def _mkLabel(i):
                return {'1.':f'{i+iOff}.','I.':f'{roman.toRoman(i+iOff)}.','i.':f'{roman.toRoman(i+iOff).lower()}.','(i)':f'({roman.toRoman(i+iOff).lower()})','(a)':f'({(string.ascii_lowercase+string.ascii_uppercase)[i+iOff-1]})'}[e.get('labels','1.')]
            ret='\n'
            for i,li in enumerate(e):
                label=_mkLabel(i)
                ret+=f'\n\n{label} '+self.recurse(li)
            ret+='\n'
            return ret
        elif e.tag=='bibliography':
            ret='\n\n'
            for bi in e:
                if bi.tag=='bibentry':
                    ret+=f'\n.. [{bi.get("abbrev").replace(".","")}] '+self.recurse(bi).replace('\n','\n     ')
                elif bi.tag=='bibintertitle':
                    assert len(bi)==0
                    ret+='\n\n'+bi.text+'\n'+len(bi.text)*'~'+'\n'
                else: assert False
            return ret
        raise RuntimeError(f'Unhandled tag <{e.tag}>')
os.makedirs('build/sphinx/source',exist_ok=True)
writer=SphinxWriter(outdir='build/sphinx/source')
writer.write(book)
writer.write(index)
writer.write(gloss)

os.makedirs('build/sphinx-vimm/source',exist_ok=True)
writerVimm=SphinxWriter(outdir='build/sphinx-vimm/source',vimm=True)
vimmBook=etree.parse(vimmXML,etree.XMLParser(remove_comments=True)).getroot() 
writerVimm.write(vimmBook)

→ build/sphinx/source/front.rst
   → build/sphinx/source/front-A.rst
   → build/sphinx/source/front-B.rst
   → build/sphinx/source/front-C.rst
→ build/sphinx/source/part-I.rst
   → build/sphinx/source/ch-01.rst
   → build/sphinx/source/ch-02.rst
→ build/sphinx/source/part-II.rst
   → build/sphinx/source/ch-03.rst
   → build/sphinx/source/ch-04.rst
   → build/sphinx/source/ch-05.rst
   → build/sphinx/source/ch-06.rst
   → build/sphinx/source/ch-07.rst
   → build/sphinx/source/ch-08.rst
   → build/sphinx/source/ch-09.rst
   → build/sphinx/source/ch-10.rst
   → build/sphinx/source/ch-11.rst
   → build/sphinx/source/ch-12.rst
   → build/sphinx/source/ch-13.rst
→ build/sphinx/source/part-III.rst
   → build/sphinx/source/ch-14.rst
   → build/sphinx/source/ch-15.rst
   → build/sphinx/source/ch-16.rst
   → build/sphinx/source/ch-17.rst
   → build/sphinx/source/ch-18.rst
   → build/sphinx/source/ch-19.rst
   → build/sphinx/source/ch-20.rst
   → build/sphinx/source/ch-21.rst
   → build/sphinx/so

In [ ]:
## DocBook
from lxml import etree
import os

book=etree.parse('build/book.sectioned.xml',etree.XMLParser(remove_blank_text=True)).getroot()
index=etree.parse('src/index.xml',etree.XMLParser(remove_blank_text=True)).getroot() 
gloss=etree.parse('src/gloss.xml',etree.XMLParser(remove_blank_text=True)).getroot()
bookMetadata=etree.parse('docbook/metadata.xml',etree.XMLParser(remove_blank_text=True)).getroot()
book.append(index)
book.append(gloss)

vimmBook=etree.parse(vimmXML,etree.XMLParser(remove_blank_text=True)).getroot()
# bibDocbook=etree.parse('docbook/bib.xml',etree.XMLParser(remove_blank_text=True)).getroot()

frontmatter=False

def _docb_writer(e,ord=0,parent=None,xslTNG=False):
    xlinkNs='http://www.w3.org/1999/xlink'
    xmlNs='http://www.w3.org/XML/1998/namespace'
    xmlPrefix='{'+xmlNs+'}'
    editorial_titles=False
    
    if xslTNG:
        xlinkPrefix='{'+xlinkNs+'}'
        nsmap={None:'http://docbook.org/ns/docbook','xlink':xlinkNs,'xml':xmlNs}
    else:
        #nsmap={None:'http://docbook.org/ns/docbook','xml':xmlNs}
        #xlinkPrefix=''
        nsmap,xlinkPrefix=None,''

    def _E(e,text=None,*,subs=[],xml_id=None,**kw):
        assert text is None or isinstance(text,str)
        ret=etree.Element(e,**kw,nsmap=nsmap) # ,'pub':pubNs})
        if xml_id is not None:
            # if xslTNG:
            ret.attrib[xlinkPrefix+'label']=xml_id
            ret.attrib[xmlPrefix+'id']=xml_id
        if 'linkend' in kw:
            if xslTNG:
                ret.attrib[xlinkPrefix+'to']=kw['linkend']
                kw.pop('linkend')
            pass
        ret.text=text
        for sub in subs:
            if sub is None: continue
            if sub.tag=='__FLATTEN__':
                for su in sub: ret.append(su)
            else: ret.append(sub)
        return ret

    def _curr_chapter(e):
        while (e:=e.getparent()) is not None:
            if e.tag=='struct-2-chapter':
                heading=e[0]
                assert heading.tag.startswith('heading')
                return heading.attrib.get('toc_num',None)


    def _recurse(e,dbg=False):
        if e.text is not None and len(e)==0:
            if dbg: print('=',e.text)
            return [_E('phrase',e.text)]
        if e.text is not None and e.text!='' and len(e)>0:
            print('$$$$',e.tag,e.sourceline) # ,len(e.text),e.text)
        ret=[_docb_writer(e2,ord=ord,parent=e) for ord,e2 in enumerate(e)]
        if dbg: print('|'.join([r.text for r in ret if r.text]))
        return ret
    if isinstance(e,etree._Comment): return None
    if e.tag=='book':
        # return _E('book',xmlns='http://docbook.org/ns/docbook',version="5.0",subs=_recurse(e))
        b=_E('book',version="5.2",subs=[bookMetadata]+_recurse(e))
        # b.append(bibDocbook)
        return b
        # b.attrib['xmlns:pub']="http://docbook.org/ns/docbook/publishers"
    if e.tag=='em':
        assert len(e)==0
        return _E('emphasis',e.text)  
    elif e.tag=='span':
        assert len(e)==0
        if e.text is None: return None
        if (fam:=e.attrib.get('family',None)) is None: return _E('phrase',e.text)
        elif fam=='italic': return _E('emphasis',e.text)
        elif fam=='bold': return _E('emphasis',e.text,role='bold')
        elif fam=='smallcaps': return _E('emphasis',e.text,role='smallcaps')
        elif fam=='bold-italic': return _E('emphasis',e.text,role='bold-italic')
        else: raise RuntimeError(f'Unrecognized family {fam}')
    elif e.tag=='p':
        if not editorial_titles and 'editorial_title' in e.attrib: return None
        if 'anchor' in e.attrib:
            return _E('formalpara',xml_id=e.attrib['anchor'],subs=[_E('title','§'+e.attrib['num']),_E('para',subs=_recurse(e))])
        else: return _E('para',subs=_recurse(e))
    elif e.tag=='footnote':
        if (ch:=_curr_chapter(e)) is None:
            assert 'reference_existing_footnote' not in e.attrib
            return _E('footnote',subs=_recurse(e))
        else:
            label=f'{ch}.n{e.attrib["mark"]}'
            if 'reference_existing_footnote' in e.attrib: return _E('footnoteref',linkend=label)
            return _E('footnote',subs=_recurse(e),xml_id=label)
    elif e.tag=='verse':
        for ich,ch in enumerate(e):
            if not ch.tag=='line':
                raise RuntimeError(f'<verse> may contain only <line> elements (<verse> at line {e.sourceline}; child #{ich} <{ch.tag}>, line {ch.sourceline})')
        return _E('linegroup',subs=[_E('speaker')]+_recurse(e))
        return _E('linegroup',subs=[_E('speaker')]+_recurse(e))
        # return _E('poetry',subs=[_E('linegroup',subs=_recurse(e))])
    elif e.tag=='line': return _E('line',subs=_recurse(e))
    elif e.tag.startswith('struct-'):
        level,tail=int((m:=re.match('struct-(?P<tail>(?P<level>[0-9]+)-.*)',e.tag)).group('level')),m.group('tail')
        heading=e[0]
        assert heading.tag.startswith('heading')
        toc_num=e.attrib.get('toc_num',None)
        title=[_E('title',subs=_recurse(heading))]
        if level==1: # part
            global frontmatter
            frontmatter=((name:=e.attrib['name'])=='(Front)')
            if frontmatter: return _E('__FLATTEN__',subs=_recurse(e))
            kw=({'xml_id':'part-'+toc_num} if toc_num else {})
            return _E('part',subs=title+_recurse(e),**kw)
        elif level==2:
            if frontmatter: return _E('preface',subs=title+_recurse(e))
            if sub:=heading.get('subtitle_pali',None): title+=[_E('subtitle',subs=[_E('phrase',subs=[_E('emphasis',text=sub)])])]
            return _E('chapter',subs=title+_recurse(e),xml_id=toc_num)
        else: return _E('section',subs=title+_recurse(e))
    elif e.tag.startswith('heading'): return None
    # elif e.tag in ('struct-3-section','struct-4-subsection','struct-5-subsubsection','struct-6-subsubsubsection','struct-7-subsubsubsubsection','struct-8-subsubsubsubsubsection'): return _E('section',subs=_recurse(e))
    elif e.tag=='printed_page':
        # return None # XXXXX
        if e.attrib['edition'] in ('BPS2011','BPS1995'): return _E('literal',f'[{e.text}|{e.attrib["page_id"]}]')
        elif e.attrib['edition']=='PTS': return _E('varname',f'({e.text})')
    elif e.tag=='ref':
        if e.attrib['type']=='vism': return _E('link',e.text,linkend=e.attrib['target'])
        elif e.attrib['type']=='bib':
            if 'loc' in e.attrib: return _E('phrase',subs=[_E('citation',e.attrib['target']),_E('phrase',' '+e.attrib['loc'])])
            return _E('citation',e.attrib['target'])
        assert False
    elif e.tag=='index': return _E('index',subs=[_E('title',e.attrib['title']),_E('subtitle',e.attrib['subtitle'])]+_recurse(e))
    elif e.tag=='glossary':  return _E('glossary',subs=[_E('title',e.attrib['title']),_E('subtitle',e.attrib['subtitle'])]+_recurse(e))
    elif e.tag=='introductory': return _E('para',subs=_recurse(e))
    elif e.tag=='entry':
        if parent.tag=='index': return _E('primaryie',subs=[_E('emphasis',text=e.attrib['title'],role='bold')]+_recurse(e))
        elif parent.tag=='glossary': return _E('glossentry',subs=[_E('glossterm',text=e.attrib['title'])]+[_E('glossdef',subs=_recurse(e))])
        print(parent.tag)
        assert False
    elif e.tag=='table':
        hasHead=(e.attrib['layout_type']!='consciousness')
        tab=_E('informaltable',subs=([_E('thead')] if hasHead else [])+[_E('tbody')])
        for irow,tr in enumerate(e):
            assert tr.tag=='tr' and set([td.tag for td in tr])=={'td'}
            row=_E('tr',subs=[_E('td',subs=(_recurse(td) if len(td)>0 else []),valign='top') for td in tr])
            tab[min(irow,1) if hasHead else 0].append(row)
        return tab
    elif e.tag=='enum':
        enum=_E('orderedlist',numeration={'1.':'arabic','I.':'upperroman','i.':'lowerroman','(i)':'lowerroman','(a)':'loweralpha'}[e.get('labels','1')])
        if 'start' in e.attrib: enum.attrib['startingnumber']=e.attrib['start']
        for li in e:
            enum.append(_E('listitem',subs=_recurse(li)))
            # if 'label' in e.attrib: print('Ignoring custom label "{e.attrib["label"]}" (line {li.sourceline})')
        return enum
    elif e.tag=='bibliography':
        bib0=_E('bibliography',subs=[bib:=_E('bibliodiv')])
        for bi in e:
            if bi.tag=='bibentry': bib.append(_E('biblioentry',subs=[_E('abbrev',bi.attrib['abbrev']),_E('title',subs=_recurse(bi))]))
            elif bi.tag=='bibintertitle': bib0.append(bib:=_E('bibliodiv',subs=[_E('title',bi.text)]))
            else: raise ValueError(f'Unhandled tag in bibliography: {bi.tag} {bi.sourceline}')
            # else: assert False
        return bib0
    elif e.tag=='raw':
        return etree.parse('docbook/'+e.attrib['file']+'.xml',etree.XMLParser(remove_blank_text=True)).getroot()
    elif e.tag=='TODO':
        return _E('emphasis','[TODO: '+e.text+']',role='bold')
    elif e.tag=='IGNORE': return None
    raise RuntimeError(f'Unhandled tag <{e.tag}>')

def _fix_formalpara(book):
    for fp in book.findall('.//formalpara'):
        subs=[]
        p=fp
        while ((p:=p.getnext()) is not None) and (p.tag in ('para','linegroup')):
            subs.append(p)
        for p in subs: fp.append(p)
    return book
            
for xslTNG in True,False:
    for stem,bk in [('vism',book),('vimm',vimmBook)]:
        docb=_docb_writer(bk,xslTNG=xslTNG)
        docb=_fix_formalpara(docb)
        kw=dict(doctype=None,xml_declaration=True,encoding='utf-8')
        os.makedirs('build/docbook',exist_ok=True)
        if xslTNG: stem+='.xslTNG'
        # stem='vism.xslTNG' if xslTNG else 'vism'
        open(f'build/docbook/{stem}.xml','wb').write(etree.tostring(docb,pretty_print=False,**kw))
        open(f'build/docbook/{stem}.pretty.xml','wb').write(etree.tostring(docb,pretty_print=True,**kw))
        print(f'→ build/docbook/{stem}.xml build/docbook/{stem}.pretty.xml')

In [23]:
## HTML5
from lxml import etree
import os
import re

book=etree.parse('build/book.sectioned.xml',etree.XMLParser(remove_blank_text=True)).getroot()
index=etree.parse('src/index.xml',etree.XMLParser(remove_blank_text=True)).getroot() 
gloss=etree.parse('src/gloss.xml',etree.XMLParser(remove_blank_text=True)).getroot()
# bibHtml5=etree.parse('html5/bib.html5',etree.XMLParser(remove_blank_text=True)).getroot()
# TODO: bibliography

vimmBook=etree.parse(vimmXML,etree.XMLParser(remove_blank_text=True)).getroot()

frontmatter=False

def _html5_writer(e,ord=0,parent=None):
    editorial_titles=False
    
    def _E(e,text=None,*,subs=[],xml_id=None,**kw):
        assert text is None or isinstance(text,str)
        if 'class_' in kw: kw['class']=kw.pop('class_')
        ret=etree.Element(e,**kw) #,nsmap=nsmap) # ,'pub':pubNs})
        if xml_id is not None: ret.attrib['id']=xml_id
        ret.text=text
        assert not isinstance(subs,etree._Element)
        assert isinstance(subs,list)
        for sub in subs:
            if sub is None: continue
            if sub.tag=='__FLATTEN__':
                for su in sub: ret.append(su)
            else: ret.append(sub)
        return ret

    def _curr_chapter(e):
        while (e:=e.getparent()) is not None:
            if e.tag=='struct-2-chapter':
                assert e[0].tag.startswith('heading')
                return e[0].get('toc_num',None)

    def _recurse(e,dbg=False):
        if e.text is not None and len(e)==0:
            if dbg: print('=',e.text)
            return [_E('span',e.text)]
        if e.text is not None and e.text!='' and len(e)>0:
            print('$$$$',e.tag,e.sourceline) # ,len(e.text),e.text)
            assert False
        ret=[_html5_writer(e2,ord=ord,parent=e) for ord,e2 in enumerate(e)]
        if dbg: print('|'.join([r.text for r in ret if r.text]))
        return ret
    if isinstance(e,etree._Comment): return None
    if e.tag=='book':
        # _E('head',subs=[_E('link',rel='stylesheet',href='style.A4.css')]),
        return _E('html',subs=[_E('body',subs=_recurse(e))],lang='en')
    if e.tag=='em':
        assert len(e)==0
        return _E('em',e.text)  
    elif e.tag=='span':
        assert len(e)==0
        if e.text is None: return None
        if (fam:=e.attrib.get('family',None)) is None: return _E('span',e.text)
        elif fam=='italic': return _E('em',e.text)
        elif fam=='bold': return _E('strong',e.text)
        elif fam=='smallcaps': return _E('span',e.text,style='font-variant: small-caps;')
        elif fam=='bold-italic': return _E('strong',e.text)
        else: raise RuntimeError(f'Unrecognized family {fam}')
    elif e.tag=='p':
        if not editorial_titles and 'editorial_title' in e.attrib: return None
        if anchor:=e.get('anchor',None):
            kw=dict(id=anchor,class_='vism-para')
            pre=[_E('strong','§'+e.attrib['num']+'. ')]
        else: kw,pre={},[]
        return _E('p',subs=pre+_recurse(e),**kw)
    elif e.tag=='footnote':
        if (ch:=_curr_chapter(e)) is None:
            assert 'reference_existing_footnote' not in e.attrib
            return _E('span',subs=_recurse(e),class_='footnote')
        else:
            label=f'{ch}.n{e.attrib["mark"]}'
            if 'reference_existing_footnote' in e.attrib:
                return _E('span',f'FIXME: see the other footnote {label}.',class_='footnote')
            return _E('span',subs=_recurse(e),id=label,class_='footnote')
    elif e.tag=='verse':
        for ich,ch in enumerate(e):
            if not ch.tag=='line':
                raise RuntimeError(f'<verse> may contain only <line> elements (<verse> at line {e.sourceline}; child #{ich} <{ch.tag}>, line {ch.sourceline})')
        return _E('div',subs=_recurse(e),class_='vism-verse')
        # return _E('poetry',subs=[_E('linegroup',subs=_recurse(e))])
    elif e.tag=='line': return _E('div',subs=_recurse(e))
    elif e.tag.startswith('struct-'):
        level,tail=int((m:=re.match('struct-(?P<tail>(?P<level>[0-9]+)-.*)',e.tag)).group('level')),m.group('tail')
        heading=e[0]
        assert heading.tag.startswith('heading')
        toc_num=heading.attrib.get('toc_num',None)
        title=[_E('h1',subs=_recurse(heading))]
        if level==1: # part
            global frontmatter
            frontmatter=((name:=e.attrib['name'])=='(Front)')
            if frontmatter: return _E('__FLATTEN__',subs=_recurse(e))
            kw=({'xml_id':'part-'+toc_num} if toc_num else {})
            return _E('section',subs=title+_recurse(e),class_='sect-dp1',**kw)
        elif level==2:
            if frontmatter: return _E('section',subs=title+_recurse(e),class_='sect-dp2')
            if sub:=heading.get('subtitle_pali',None): title+=[_E('div',role='doc-subtitle',subs=[_E('span',subs=[_E('emphasis',text=sub)])])]
            return _E('section',subs=title+_recurse(e),xml_id=toc_num,class_='sect-dp2')
        else: return _E('section',subs=title+_recurse(e),class_=f'sect-dp{level}')
    elif e.tag.startswith('heading'): return None
    elif e.tag=='printed_page':
        if e.attrib['edition'] in ('BPS2011','BPS1995'): return _E('span',f'[{e.text}|{e.attrib["page_id"]}]',class_='vism-page-bps')
        elif e.attrib['edition']=='PTS': return _E('span',f'({e.text})',class_='vism-page-pts')
    elif e.tag=='ref':
        target=e.attrib['target']
        if e.attrib['type']=='vism': return _E('a',e.text,href='#'+target)
        elif e.attrib['type']=='bib':
            ret=_E('span',class_='vism-bibref',subs=[_E('a',text=target,href='#bib:'+target)])
            if loc:=e.get('loc',None):
                if href:=e.get('href',None): ret.append(_E('a',text=' '+loc,href=href))
                else: ret.append(_E('span',' '+loc))
            return ret  
        assert False
    elif e.tag in ('index','glossary'):
        return _E('section',class_='sect-dp2',subs=[_E('h1',e.attrib['title']),_E('h2',e.attrib['subtitle']),_E('dl',class_='cols-2',subs=_recurse(e))])
    elif e.tag=='introductory': return _E('p',subs=_recurse(e))
    elif e.tag=='entry':
        return _E('__FLATTEN__',subs=[_E('dt',text=e.attrib['title']),_E('dd',subs=_recurse(e))])
    elif e.tag=='table':
        tab=_E('table',subs=[_E('thead'),_E('tbody')])
        for irow,tr in enumerate(e):
            assert tr.tag=='tr' and set([td.tag for td in tr])=={'td'}
            row=_E('tr',subs=[_E('td',subs=_recurse(td),valign='top') for td in tr])
            tab[min(irow,1)].append(row)
        return tab
    elif e.tag=='enum':
        enum=_E('ol',
            style='list-style-type: '+{'1.':'decimal','I.':'upper-roman','i.':'lower-roman','(i)':'lower-roman','(a)':'lower-alpha'}[e.get('labels','1')]
        )
        if 'start' in e.attrib: enum.attrib['start']=e.attrib['start']
        for li in e:
            enum.append(_E('li',subs=_recurse(li)))
        return enum
    elif e.tag=='bibliography':
        bib0=_E('__FLATTEN__',subs=[bib:=_E('dl',class_='bibliography')])
        for bi in e:
            if bi.tag=='bibentry':
                bib.append(_E('dt',text=bi.attrib["abbrev"],id=f'bib:{bi.attrib["abbrev"]}'))
                bib.append(_E('dd',subs=_recurse(bi)))
            elif bi.tag=='bibintertitle':
                bib0.append(_E('section',subs=[_E('h1',bi.text),bib:=_E('dl',class_='bibliography')]))
            else: assert False
        return bib0
    elif e.tag=='raw':
        return etree.parse('html5/'+e.attrib['file']+'.xml',etree.XMLParser(remove_blank_text=True)).getroot()
    elif e.tag=='IGNORE': return None
    elif e.tag=='TODO':
        return _E('strong','[TODO: '+e.text+']',class_='vism-todo')
    raise RuntimeError(f'Unhandled tag <{e.tag}> (line {e.sourceline})')


if 1:
    book.append(index)
    book.append(gloss)
    os.makedirs('build/html5',exist_ok=True)
    for stem,bk in [('vism',book),('vimm',vimmBook)]:
        ht=_html5_writer(bk)
        open(f'build/html5/{stem}.book.xml','wb').write(etree.tostring(bk,pretty_print=True))
        kw=dict(doctype=None,xml_declaration=True,encoding='utf-8',method='xml')
        os.makedirs('build/html5',exist_ok=True)
        open(f'build/html5/{stem}.html','wb').write(etree.tostring(ht,**kw))
        open(f'build/html5/{stem}.pretty.html','wb').write(etree.tostring(ht,pretty_print=True,**kw))
        print(f'→ build/html5/{stem}.html build/html5/{stem}.pretty.html')

→ build/html5/vism.html build/html5/vism.pretty.html
→ build/html5/vimm.html build/html5/vimm.pretty.html
